In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from sklearn.impute import SimpleImputer
%matplotlib inline

# 資料清洗

In [ ]:
"""
欄位所表達的資訊是否符合其資料型態  比如說應該用樹值表示卻變成object
檢查缺失值是否用np.nan表示
"""

In [ ]:
# Replace all occurrences of Not Available with numpy not a number
data = data.replace({'Not Available': np.nan})

# Iterate through the columns
for col in list(data.columns):
    # Select columns that should be numeric
    if ('ft²' in col or 'kBtu' in col or 'Metric Tons CO2e' in col or 'kWh' in 
        col or 'therms' in col or 'gal' in col or 'Score' in col):
        # Convert the data type to float
        data[col] = data[col].astype(float)

In [ ]:
"""
資料缺失大於50% 的直接去掉  救不回來
"""

# Get the columns with > 50% missing
missing_df = missing_values_table(data);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('We will remove %d columns.' % len(missing_columns))

In [ ]:
"""
把 某col 為null 的單drop, 返回得都是有該col 的欄位
"""
data.dropna(subset=['score'])

In [ ]:
"""
使用SimpleImputer
單一series 缺失職填充
注意只能用於numeric data
注意填充完會是float64 type

"""
def SimpleImputer_one_sol(df, col_name, s = "median"):
    imp = SimpleImputer(missing_values=np.nan, strategy = s)
    new_data = imp.fit_transform(df[col_name].values.reshape(-1,1))
    df[col_name] = new_data
    return df

## 離群點

In [ ]:
"""
直接觀看
data['Site EUI (kBtu/ft²)'].dropna().sort_values().tail(10)
"""
data['Site EUI (kBtu/ft²)'].describe()


### 剔除离群点

离群点的选择可能需要再斟酌一些，这里选择的方法是extreme outlier。

*   $\text{First Quartile} -3 * \text{Interquartile Range}$
*   $\text{Third Quartile} + 3 * \text{Interquartile Range}$

In [ ]:
"""
剔除單一col的離群點 c = 1.5 or 3
"""
def remove_outliners(data, col, c):
    first_quartile = data[col].describe()['25%']
    third_quartile = data[col].describe()['75%']
#     print(first_quartile)
#     print(third_quartile)
    # Interquartile range
    iqr = third_quartile - first_quartile
#     print(iqr)
    # Remove outliers
    data = data[(data[col] > (first_quartile - c * iqr)) &
                (data[col] < (third_quartile + c * iqr))]
#     print(data)
    return data

# 資料缺失

In [ ]:
# msno.bar(my_train)  #cabin and age col 都有缺失
msno.matrix(my_all_data,
            figsize = (16,7),
            width_ratios = (15,1)
           )
# msno.heatmap(my_train)   #缺失之間並沒有相關性

In [ ]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

# 圖形化展示特徵 EDA

In [ ]:
"""
單變量展示  建議先檢查Y
"""
def show_one_col(data,col):
    figsize(8, 8)
    # Histogram of the Energy Star Score
    plt.style.use('fivethirtyeight')
    plt.hist(data[col].dropna(), bins = 100, edgecolor = 'k');
    plt.xlabel('Score'); plt.ylabel("count of {}".format(col)); 
    plt.title(col);

In [ ]:
"""
用一離散特徵當類別  連續特徵當Y值  查看連續特徵的離群點
"""
def box_des_con(data, col_des, col_conti):
    fig = sns.boxplot(x = col_des, y = col_conti, data = data)
    fig.axis(ymin = data[col_conti].min(), ymax = data[col_conti].max())


In [1]:
"""
看單一Col裡面所有離散值 對Y的影響  分布越不規則  表示該特徵越重要 
"""
def one_col_feature_to_Y(data, col, target, threshold = 100):
    types = data.dropna(subset=[target])
    types = types[col].value_counts()
    types = list(types[types.values > threshold].index)
    figsize(12, 10)

    # Plot each building
    for b_type in types:
        # Select the building type
        subset = data[data[col] == b_type]

        # Density plot of Energy Star scores
        sns.kdeplot(subset[target].dropna(),
                   label = b_type, shade = False, alpha = 0.8);

    # label the plot
    plt.xlabel(target, size = 20); plt.ylabel('Density', size = 20); 
    plt.title('Density Plot of {} by {}'.format(target,col), size = 28);

In [ ]:
"""
將數值資料做非線性變換  cat資料做one hot, 測試跟Y的相關性會不會提升
注意cat 欄位的塞選
correlations, features = EDA_transform_test_data(data, ['Borough', 'Largest Property Use Type'], "score")
"""
def EDA_transform_test_data(data, catagory_cols, target):
    numeric_subset = data.select_dtypes('number')

# Create columns with square root and log of numeric columns
    for col in numeric_subset.columns:
        # Skip the Energy Star Score column
        if col == target:
            next
        else:
            numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
            numeric_subset['log_' + col] = np.log(numeric_subset[col])

    # Select the categorical columns
    categorical_subset = data[catagory_cols]

    # One hot encode
    categorical_subset = pd.get_dummies(categorical_subset)

    # Join the two dataframes using concat
    # Make sure to use axis = 1 to perform a column bind
    features = pd.concat([numeric_subset, categorical_subset], axis = 1)

    # Drop buildings without an energy star score
    features = features.dropna(subset = [target])

    # Find correlations with the score 
    correlations = features.corr()[target].dropna().sort_values()
    return correlations, features

In [ ]:
"""
雙變數展示
"""
figsize(12, 10)

# Extract the building types
features['Largest Property Use Type'] = data.dropna(subset = ['score'])['Largest Property Use Type']

# Limit to building types with more than 100 observations (from previous code)
features = features[features['Largest Property Use Type'].isin(types)]

# Use seaborn to plot a scatterplot of Score vs Log Source EUI
sns.lmplot('Site EUI (kBtu/ft²)', 'score', 
          hue = 'Largest Property Use Type', data = features,
          scatter_kws = {'alpha': 0.8, 's': 60}, fit_reg = False,
          size = 12, aspect = 1.2);

# Plot labeling
plt.xlabel("Site EUI", size = 28)
plt.ylabel('Energy Star Score', size = 28)
plt.title('Energy Star Score vs Site EUI', size = 36);

In [ ]:
"""
pair_plot
"""
# Extract the columns to  plot
plot_data = features[['score', 'Site EUI (kBtu/ft²)', 
                      'Weather Normalized Source EUI (kBtu/ft²)', 
                      'log_Total GHG Emissions (Metric Tons CO2e)']]

# Replace the inf with nan
plot_data = plot_data.replace({np.inf: np.nan, -np.inf: np.nan})

# Rename columns 
plot_data = plot_data.rename(columns = {'Site EUI (kBtu/ft²)': 'Site EUI', 
                                        'Weather Normalized Source EUI (kBtu/ft²)': 'Weather Norm EUI',
                                        'log_Total GHG Emissions (Metric Tons CO2e)': 'log GHG Emissions'})

# Drop na values
plot_data = plot_data.dropna()

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, size = 3)

# Upper is a scatter plot
grid.map_upper(plt.scatter, color = 'red', alpha = 0.6)

# Diagonal is a histogram
grid.map_diag(plt.hist, color = 'red', edgecolor = 'black')

# Bottom is correlation and density plot
grid.map_lower(corr_func);
grid.map_lower(sns.kdeplot, cmap = plt.cm.Reds)

# Title for entire plot
plt.suptitle('Pairs Plot of Energy Data', size = 36, y = 1.02);